# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [1]:
import findspark
findspark.init('/home/dhrumpy/spark-3.0.0-bin-hadoop3.2')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')
data.show(5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows



In [8]:
# re;abeling the column
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



## Clean and Prepare the Data

** Create a new length feature: **

In [9]:
from pyspark.sql.functions import length # similar to the user defined function defined in the earlier notebook

In [12]:
data = data.withColumn('length',length(data['text']))
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [14]:
# Pretty Clear Difference
# feature engineering
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [15]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [16]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [17]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [18]:
from pyspark.ml.classification import NaiveBayes

In [19]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [20]:
from pyspark.ml import Pipeline

In [25]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [26]:
cleaner = data_prep_pipe.fit(data)

In [27]:
clean_data = cleaner.transform(data)
clean_data.show(5)

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

### Training and Evaluation!

In [28]:
clean_data = clean_data.select(['label','features'])

In [30]:
clean_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows



In [31]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [32]:
spam_predictor = nb.fit(training)

In [33]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [34]:
test_results = spam_predictor.transform(testing)

In [37]:
test_results.show(5)  # we need to compare the label versus the prediction

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-807.81308928963...|[1.0,2.6475991840...|       0.0|
|  0.0|(13424,[0,1,2,13,...|[-606.47967921602...|[1.0,2.8640306625...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-576.27663915292...|[1.0,6.7993796294...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-823.53939967014...|[1.0,8.4838779418...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-883.17331788431...|[1.0,2.1346254784...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [38]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [39]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9148299223577947
